# Exploração de Dados, Pré-processamento e Levantamento de Hipóteses

# Seção 4.2 (SPRINT 2)


Este notebook documenta a análise e transformação dos dados realizada durante a sprint. Serão apresentados os processos de exploração, pré-processamento dos dados e as hipóteses geradas a partir da análise.


## 4.2.1. Exploração de Dados (EDA)

Nesta seção, será apresentada uma análise detalhada das características das variáveis que compõem o conjunto de dados. O objetivo é identificar as principais tendências, padrões e possíveis inconsistências, facilitando o entendimento do comportamento dos dados. Esta análise é crucial para direcionar as etapas subsequentes do desenvolvimento do modelo preditivo.


### Carregando os Dados


#### Instalando bibliotecas


In [ ]:
%pip install numpy pandas matplotlib seaborn scikit-learn scipy

#### Fazendo o merge das tabelas


In [ ]:
# Carregando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import time
import datetime
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
from sklearn.preprocessing import PowerTransformer

generate_plots = True  # Mude caso queira regerar todos os gráficos (pode demorar)

# Definindo os caminhos dos arquivos
file_paths = [
    "../assets/dataset/month_2.csv",
    "../assets/dataset/month_3.csv",
    "../assets/dataset/month_4.csv",
    "../assets/dataset/month_5.csv",
    "../assets/dataset/month_6.csv",
]

info_cadastral = pd.read_csv("../assets/dataset/informacao_cadastral.csv")


# Carregando os arquivos em DataFrames
dfs = [pd.read_csv(file_path) for file_path in file_paths]

# Junta todos os DataFrames em um único DataFrame por 'clientCode' e 'clientIndex'
df_mensal = pd.concat(dfs, ignore_index=True)
df = pd.merge(df_mensal, info_cadastral, on=["clientCode", "clientIndex"], how="left")

# Exibe as primeiras 5 linhas do DataFrame
df.head()

In [ ]:
print(f"Colunas no dataframe (df): \n\n{df.columns}")

### Estatística Descritiva

A seguir, são apresentadas as estatísticas descritivas básicas de cada coluna do conjunto de dados, bem como a classificação das variáveis em numéricas ou categóricas.


#### Descrição Básica


In [ ]:
# Mostra o tamanho e a forma do DataFrame
print(
    f"Formato do DataFrame (linha, colunas): \nLinhas: {df.shape[0]} \nColunas: {df.shape[1]}"
)

# Estatísticas descritivas do DataFrame (média, desvio padrão, mínimo, máximo, etc)
print("\nEstatísticas descritivas:")
df.describe().apply(lambda x: x.apply("{:.2f}".format))

#### Analisando valores nulos


In [ ]:
import matplotlib.pyplot as plt

# Calcula a porcentagem de valores ausentes em cada coluna
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Ordena as colunas com valores ausentes em ordem decrescente
missing_percentage = missing_percentage.sort_values(ascending=False)

# Cria um gráfico de barras com as porcentagens de valores ausentes
plt.figure(figsize=(12, 6))
bars = missing_percentage.plot(kind="bar", color="salmon")
plt.title("Percentual de Valores Ausentes por Coluna")
plt.xlabel("Colunas")
plt.ylabel("Percentual de Valores Ausentes")
plt.xticks(rotation=45)

# Adiciona os números de valores nulos na diagonal
for bar, missing_count in zip(bars.patches, missing_percentage):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height(),
        f"{missing_count:.2f}%",
        ha="center",
        va="bottom",
        rotation=45,
    )

plt.show()

As colunas de geolocalização ('gatewayGeoLocation.alt', 'gatewayGeoLocation.lat' e 'gatewayGeoLocation.long') tem quase 80% de nulos


#### Tipos de dados cada coluna


In [ ]:
df.dtypes

In [ ]:
# Separando as colunas entre identificadoras, categóricas e numéricas
id_features = ["clientCode", "clientIndex", "condCode", "condIndex", "meterSN"]
categorical_features = [
    col
    for col in df.select_dtypes(include=["object"]).columns
    if col not in id_features
]
numerical_features = [
    col
    for col in df.select_dtypes(include=["int64", "float64"]).columns
    if col not in id_features
]

# Exibindo as colunas categóricas e numéricas
print(f"{len(id_features)} id features:", id_features)
print(f"{len(categorical_features)} categorical features:", categorical_features)
print(f"{len(numerical_features)} numerical features:", numerical_features)

### Explorando Variáveis Chave e Relações


#### Matriz de Correlação


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Calcula a matriz de correlação
correlation_matrix = df[numerical_features].corr()

# Cria um gráfico de calor com a matriz de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

#### Gráfico de Dispersão de meterIndex vs. initialIndex com gain como Escala de Cor


In [ ]:
plt.figure(figsize=(10, 6))
scatter = sns.scatterplot(data=df, x='meterIndex', y='initialIndex', hue='gain', palette='viridis', s=100)
plt.title('Gráfico de Dispersão de meterIndex vs. initialIndex com gain como Escala de Cor')
plt.xlabel('Meter Index')
plt.ylabel('Initial Index')

# Adiciona a legenda ao gráfico
scatter.legend(title='Gain')
plt.show()

#### Histograma de RSSI


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df["rssi"], kde=True, bins=20, color="dodgerblue")
plt.title("Distribuição de RSSI")
plt.xlabel("RSSI")
plt.ylabel("Frequency")
plt.show()

#### Scatter Plot de Gain e Pulse Count


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="gain", y="pulseCount")
plt.title("Scatter Plot de Gain vs. Pulse Count")
plt.xlabel("Gain")
plt.ylabel("Pulse Count")
plt.show()

Gain aparenta só ter valores de 0.1 e 0.01


In [ ]:
df["gain"].value_counts()

#### Scatter plot de meterIndex vs. pulseCount


In [ ]:
# Cria um gráfico de barras com a contagem de valores únicos da coluna 'gain'
plt.figure(figsize=(10, 6))
plt.scatter(df["pulseCount"], df["meterIndex"], alpha=0.5, color="blue")

# Adiciona título e rótulos aos eixos
plt.title("Scatter Plot of Pulse Count vs. Meter Index")
plt.xlabel("Pulse Count")
plt.ylabel("Meter Index")

# Exibe o gráfico
plt.show()

#### Histograma de pulseCount


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df["pulseCount"], bins=5, kde=False, color="teal")
plt.title("Histogram of Pulse Count")
plt.xlabel("Pulse Count")
plt.ylabel("Frequency")
plt.show()

In [ ]:
df["pulseCount"].describe().round(3)

#### Histograma de todas as colunas numéricas


In [ ]:
# Cria histogramas para todas as colunas numéricas
df.hist(bins=5, figsize=(30, 30), color="teal")
plt.suptitle("Distribuição de colunas numéricas", fontsize=16)
plt.show()

#### Gráfico de Regressão de pulseCount vs. rssi


In [ ]:
if generate_plots:
    plt.figure(figsize=(10, 6))
    sns.regplot(data=df, x='pulseCount', y='rssi', ci=95, scatter_kws={'color':'blue'}, line_kws={'color':'red'})
    plt.title('Gráfico de Regressão de pulseCount vs. RSSI')
    plt.xlabel('Pulse Count')
    plt.ylabel('RSSI')
    plt.show()

#### Gráfico de regressão de pulseCount vs. meterIndex


In [ ]:
if generate_plots:
    plt.figure(figsize=(10, 6))
    sns.regplot(data=df, x='pulseCount', y='meterIndex', ci=95, scatter_kws={'color':'blue'}, line_kws={'color':'red'})
    plt.title('Gráfico de Regressão de pulseCount vs. meterIndex')
    plt.xlabel('Pulse Count')
    plt.ylabel('meterIndex')
    plt.show()

## 4.2.2. Pré-processamento dos Dados

O pré-processamento dos dados envolve a limpeza e transformação das variáveis para preparar o conjunto de dados para a modelagem. Nesta seção, serão abordadas as seguintes etapas: tratamento de valores ausentes, identificação de outliers e criação de novas features.


### Transformação de Dados

Para melhorar a performance dos modelos preditivos, algumas variáveis foram transformadas. A seguir estão as transformações aplicadas:

- **Normalização**: A variável `gain` foi normalizada para reduzir o impacto de escalas diferentes entre as variáveis.
- **Transformação logarítmica**: Aplicada na variável `pulseCount` para lidar com a alta variabilidade e melhorar a distribuição dos dados.
- **Codificação de variáveis categóricas**: As variáveis `inputType` e `model` foram codificadas usando one-hot encoding para permitir que os modelos utilizem essas informações.


### Tratamento de valores nulos

De acordo com o gráfico de percentual de valores ausentes por coluna e a tabela de estatística descritiva, podemos concluir que os valores nulos nas colunas 'gain' e 'rssi', poderão ser substituidos por suas respectivas média, moda ou mediana, sem afetar significativamente qualquer análise.


#### Tratando o rssi


In [ ]:
rssi_mean = df["rssi"].mean()  # Calcula a média da coluna rssi
rssi_mode = df["rssi"].mode()  # Calcula a moda da coluna rssi
rssi_median = df["rssi"].median()  # Calcula a mediana da coluna rssi

print(f"Média da coluna rssi: {rssi_mean:.2f}")
print(f"Moda da coluna rssi: {rssi_mode.values[0]:.2f}")
print(f"Mediana da coluna rssi: {rssi_median:.2f}")

In [158]:
df["rssi"] = df["rssi"].replace(
    np.nan, rssi_mean
)  # Substitui os valores nulos pela média da coluna rssi

#### Tratando o gain


In [159]:
df["gain"] = df["gain"].replace(
    np.nan, 1
)  # Substitui os valores nulos pela moda da coluna gain

### Variáveis Categóricos


#### Análise


In [ ]:
print(categorical_features)

In [ ]:
# Dicionário para armazenar a cardinalidade de cada feature categórica
cardinalities = {}

# Loop através de cada feature categórica, excluindo 'datetime'
for feature in categorical_features:
    if feature != "datetime":  # Exclui a coluna 'datetime'
        cardinality = len(df[feature].unique())
        cardinalities[feature] = cardinality

# Ordena o dicionário pela cardinalidade em ordem decrescente
sorted_cardinalities = dict(
    sorted(cardinalities.items(), key=lambda item: item[1], reverse=True)
)

# Imprime a cardinalidade de cada feature em ordem decrescente
for feature, cardinality in sorted_cardinalities.items():
    print(f"Feature: {feature}, Cardinalidade: {cardinality}")

In [ ]:
# Imprime a cardinalidade e os primeiros 10 valores únicos de cada feature em ordem decrescente
for feature in sorted_cardinalities:
    unique_values = df[feature].unique()
    cardinality = sorted_cardinalities[feature]

    print(f"Feature: {feature}")
    print(f"Cardinalidade: {cardinality}")
    print(f"Primeiros 10 Valores Únicos: {unique_values[:10]}\n")

In [ ]:
df["perfil_consumo"].value_counts()  # Mostra os perfis de consumo

Percebe-se que, na verdade, há apenas 3 categorias em perfil de consumo: Aquecedor, Cocção, e Piscina.


Contratação, CEP e bairro possuem alta cardinalidades vamos ter que tratar de alguma forma diferente.

Para a contratação podemos criar uma feature de dias desde a contratação.

Para o CEP podemos remover o '-' e tratar como número, uma vez que números próximos de CEP possuem também proximidade geográfia (i.e. o CEP '22000-200' é mais próximo geográficamente de '23200-200' do que de '12200-200').
Já para o bairro


Para bairro podemos analisar o gráfico, parece fazer sentido pegar as 9 categorias mais frequentes (até Santa Cecília), pois com 1/3 da cardinalidade (9 comparado ao total de 27 bairros) corresponde a mais de 80% dos bairros.
Assim transformaremos os demais bairros em uma coluna de "outros".


#### Entedimento


As colunas apresentam baixa cardinalidade, a maior sendo 'inputType' com 9 categorias diferentes, e nenhuma sendo ordinal.
Assim usaremos OneHotEncoding (OHE) em todas elas (exceto 'perfil_consumo', 'CEP', 'bairro' e 'contratacao'), a fim de preservar ao máximo a informação contida nas variáveis.


No caso do perfil de consumo ('perfil_consumo'), é possível perceber que existem apenas 3 categorias reais (Aquecedor, Cocção e Piscina), assim criaremos 3 colunas binárias, onde 1 indicará que o cliente possui, mas não necessariamente de forma exclusiva, aquele perfil.


#### Codificação


In [164]:
# Criando df_encoded para preservar o df original
df_encoded = df.copy()

##### Transformação da data para o formato unix timestamp


In [165]:
df["datetime"] = pd.to_datetime(
    df["datetime"]
)  # Converte a coluna datetime para o formato datetime

df["timestamp"] = df["datetime"].apply(
    lambda x: x.timestamp() if pd.notnull(x) else None
)  # Cria uma nova coluna derivada da coluna datetime no DataFrame com os valores convertidos para timestamp, ignorando os valores nulos

##### Enconding Geral (OneHotEncoder)


In [ ]:
# Remover as colunas que terão um tratamento diferente
ohe_categorical_cols = [
    col
    for col in categorical_features
    if col not in ["datetime", "perfil_consumo", "cep", "bairro", "contratacao"]
]

# Aplicar OneHotEncoder nas colunas selecionadas, mantendo o prefixo da variável original
df_encoded_ohe = pd.get_dummies(
    df_encoded[ohe_categorical_cols], prefix=ohe_categorical_cols
)

# Contar quantas colunas foram geradas no processo de One-Hot Encoding
num_new_columns = df_encoded_ohe.shape[1]
num_initial_columns = len(ohe_categorical_cols)
num_total_columns_before = df_encoded.shape[1]
num_total_columns_after = df_encoded.shape[1]

# Exibir informações detalhadas sobre o processo de codificação
print(f"Quantidade inicial de colunas categóricas codificadas: {num_initial_columns}")
print(f"Número de novas colunas geradas pelo One-Hot Encoding: {num_new_columns}")
print(f"Número total de colunas antes da codificação (df): {num_total_columns_before}")
print(
    f"Número total de colunas após a codificação (df_encoded): {num_total_columns_after}"
)


# Exibir as primeiras linhas das novas colunas geradas
print("\nVisualização das novas colunas geradas:")
print(df_encoded_ohe.head())

# Manter as colunas que não foram codificadas
df_remaining = df_encoded.drop(columns=ohe_categorical_cols)

# Concatenar as colunas codificadas com as restantes do DataFrame
df_encoded = pd.concat([df_remaining, df_encoded_ohe], axis=1)

# Verificação final: Exibir as primeiras linhas do DataFrame completo
print("Primeiras linhas do DataFrame completo com todas as colunas:")
print(df_encoded.head())

##### Encoding 'contratação'


In [ ]:
# Calcular o número de valores NaN em 'contratacao'
num_nan_contratacao = df_encoded["contratacao"].isna().sum()

# Calcular o percentual de NaN em relação ao número total de linhas
percentual_nan_contratacao = (num_nan_contratacao / len(df_encoded)) * 100

# Exibir os resultados
print(f"Número de valores NaN em 'contratacao': {num_nan_contratacao}")
print(f"Percentual de valores NaN em 'contratacao': {percentual_nan_contratacao:.2f}%")

Para realizar o encoding de 'contratacao', vamos transformar a data de contratação em uma feature nova: número de dias desde a contrataçao ('dias_desde_contratacao'), tendo como data de referência o último dado disponível na base de dados, e tratando-a como 'int' daqui pra frente.

Depois, iremos inputar os dados nulos pela mediana e criar uma coluna binária 'contratacao_isnan' que persiste a inforamção de quais linhas eram nulas nessa coluna.


In [ ]:
# Converter as colunas 'contratacao' e 'datetime' para formato datetime
df_encoded["contratacao"] = pd.to_datetime(df_encoded["contratacao"], errors="coerce")
df_encoded["datetime"] = pd.to_datetime(df_encoded["datetime"], errors="coerce")

# Encontrar o último dia de contratação (ignorando NaNs)
ultimo_dia_contratacao = df_encoded["contratacao"].max()

# Encontrar o último timestamp (ignorando NaNs)
ultimo_timestamp = df_encoded["datetime"].max()

# Exibir as datas encontradas
print(f"Último dia de contratação: {ultimo_dia_contratacao}")
print(f"Último datetime: {ultimo_timestamp}")

Vamos usar o último 'datetime' (variável 'ultimo_timestamp') como data de referência para nossa feature!


In [ ]:
# Passo 1: Converter a coluna 'contratacao' para o formato datetime
df_encoded["contratacao"] = pd.to_datetime(df_encoded["contratacao"], errors="coerce")

# Passo 2: Calcular o número de dias até a data de referência
data_referencia = ultimo_timestamp  # Último timestamp encontrado
df_encoded["dias_ate_referencia"] = (
    data_referencia - df_encoded["contratacao"]
).dt.days

# Passo 3: Criar a coluna binária 'contratacao_nan' para indicar valores NaN originais
df_encoded["contratacao_nan"] = df_encoded["dias_ate_referencia"].isna().astype(int)

# Passo 4: Substituir os valores NaN pela mediana
mediana_dias = df_encoded["dias_ate_referencia"].median()
df_encoded["dias_ate_referencia"] = df_encoded["dias_ate_referencia"].fillna(
    mediana_dias
)

# Verificar a nova feature e a coluna binária
print(df_encoded[["contratacao", "dias_ate_referencia", "contratacao_nan"]].head())

# Remover a coluna original 'contratacao'
df_encoded.drop(columns=["contratacao"], inplace=True)

# Contar o número de valores 1 na coluna 'contratacao_nan'
num_contratacao_nan_ones = df_encoded["contratacao_nan"].sum()
print(f"\nNúmero de valores 1 na coluna 'contratacao_nan': {num_contratacao_nan_ones}")
print(
    f"Percentual de valores NaN em 'contratacao_nan': {num_contratacao_nan_ones/len(df_encoded):.2%}"
)

##### Encoding 'cep'


In [ ]:
# Calcular o número de valores NaN em 'cep'
num_nan_cep = df_encoded["cep"].isna().sum()

# Calcular o percentual de NaN em relação ao número total de linhas
percentual_num_nan_cep = (num_nan_cep / len(df_encoded)) * 100

# Exibir os resultados
print(f"Número de valores NaN em 'cep' pré-encoding: {num_nan_cep}")
print(f"Percentual de valores NaN em 'cep' pré-encoding: {percentual_num_nan_cep:.2f}%")

Para 'cep' vamos substituir os valores 'nan' por 0, após isso retirar o hífen e tratar os valores como 'int', conforme a explicação na seção de entendimento.


In [ ]:
# Passo 1: Remover hífens e espaços em branco do CEP
df_encoded["cep_cleaned"] = df_encoded["cep"].str.replace("-", "").str.strip()

# Passo 2: Substituir strings vazias ou espaços em branco por NaN
df_encoded["cep_cleaned"] = df_encoded["cep_cleaned"].replace("", float("nan"))

# Passo 3: Substituir NaNs por um valor adequado, por exemplo, 0
df_encoded["cep_cleaned"] = df_encoded["cep_cleaned"].fillna("0")

# Passo 4: Converter a coluna limpa para inteiro
df_encoded["cep"] = df_encoded["cep_cleaned"].astype(int)

# Passo 5: Remover a coluna 'cep_cleaned'
df_encoded = df_encoded.drop(columns=["cep_cleaned"])

# Verificar as primeiras linhas para garantir que a conversão foi bem-sucedida
print(df_encoded["cep"].value_counts())

In [ ]:
# Contar o número de ocorrências onde o CEP é igual a 0
count_cep_zero = (df_encoded["cep"] == 0).sum()

# Calcular o percentual que isso representa em relação ao total de linhas
percent_cep_zero = (count_cep_zero / len(df_encoded)) * 100

print(f"Contagem de valores onde o CEP é igual a 0 pós-encoding: {count_cep_zero}")
print(
    f"Percentual de valores onde o CEP é igual a 0 pós-encoding: {percent_cep_zero:.2f}%"
)

print(f"\nNúmero de valores NaN em 'cep' pré-encoding: {num_nan_cep}")
print(f"Percentual de valores NaN em 'cep' pré-encoding: {percentual_num_nan_cep:.2f}%")

Realizamos o encoding, mas algo não parece certo... apareceram cerca de 2 mil linhas a mais como zero em relação ao número de NaNs inicial em 'cep'.

Vamos investigar:


In [ ]:
# Remover espaços em branco e verificar se os valores são apenas '-' ou strings vazias após o strip
df["cep_cleaned"] = df["cep"].str.strip()

# Contar o número de ocorrências onde o CEP é apenas '-' ou uma string vazia
count_hyphen_or_empty = ((df["cep_cleaned"] == "-") | (df["cep_cleaned"] == "")).sum()

print(
    f"Número de linhas em df onde o CEP é apenas '-' ou uma string vazia após o strip: {count_hyphen_or_empty}"
)
print(f"Diferença não explicada pré e pós encoding: {count_cep_zero - num_nan_cep}")

Conseguimos justificar. Os casos eram preenchimentos que estavam somente com espaços ou hífen, assim, após o 'strip()' (que remove espaços) podemos ver que eram, na verdade, células não-preenchidas.

Podemos remover o df['cep_cleand'] agora.


In [174]:
df = df.drop(columns="cep_cleaned")

##### Encoding 'bairro'


In [ ]:
df_encoded["bairro"].value_counts()

In [ ]:
# Calcular o número de valores NaN em 'bairro'
num_nan_bairro = df_encoded["bairro"].isna().sum()

# Calcular o percentual de NaN em relação ao número total de linhas
percentual_num_nan_bairro = (num_nan_bairro / len(df_encoded)) * 100

# Exibir os resultados
print(f"Número de valores NaN em 'bairro' pré-encoding: {num_nan_bairro}")
print(
    f"Percentual de valores NaN em 'bairro' pré-encoding: {percentual_num_nan_bairro:.2f}%"
)

In [ ]:
# Passo 1: Calcular a frequência de cada categoria na coluna 'bairro' e ordenar
frequencia_bairro = df_encoded["bairro"].value_counts().sort_values(ascending=False)

# Calcular a frequência acumulada
frequencia_acumulada = frequencia_bairro.cumsum()

# Normalizar a frequência acumulada para obter a proporção (entre 0 e 1)
frequencia_acumulada = frequencia_acumulada / frequencia_acumulada.max()

# Passo 2: Plotar a frequência acumulada
plt.figure(figsize=(12, 6))
plt.plot(frequencia_acumulada, marker="o", linestyle="-", color="b")
plt.title('Frequência Acumulada de "Bairro" do Maior para o Menor')
plt.xlabel("Bairro")
plt.ylabel("Frequência Acumulada (Normalizada)")
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

Para bairro podemos analisar o gráfico, parece fazer sentido pegar as 9 categorias mais frequentes (até Santa Cecília), pois com 1/3 da cardinalidade (9 comparado ao total de 27 bairros) corresponde a mais de 80% dos bairros.
Assim transformaremos os demais bairros em uma coluna de "outros".


In [ ]:
# Obter os 9 principais bairros
top_9_bairros = frequencia_bairro.head(9).index

# Passo 2: Substituir os bairros que não estão entre os 9 principais por "Outros"
df_encoded["bairro"] = df_encoded["bairro"].apply(
    lambda x: x if x in top_9_bairros else "Outros"
)

# Passo 3: Verificar o resultado
print(df_encoded["bairro"].value_counts())

In [ ]:
# Contar o número de ocorrências de "Outros"
outros_count = df_encoded["bairro"].value_counts()["Outros"]

# Contar o total de registros
total_count = len(df_encoded)

# Calcular o percentual de "Outros" em relação ao total
percent_outros = (outros_count / total_count) * 100

# Imprimir o resultado
print(f"Número de registros como 'Outros': {outros_count}")
print(f"Percentual de 'Outros' em relação ao total: {percent_outros:.2f}%")

'Outros' é 20% dos casos, como queríamos, a diferença em relação ao gráfico é por causa dos 3.97% de casos nulos. Esses casos não serão 'True' (1) em nenhuma das colunas binárias após o OHE, assim, preservaremos a informação de queram nulos.


Agora só precisamos fazer o OHE das categorias restantes:


In [180]:
# Passo 1: Aplicar o One-Hot Encoding na coluna 'bairro'
bairro_dummies = pd.get_dummies(df_encoded["bairro"], prefix="bairro")

# Passo 2: Remover a coluna original 'bairro'
df_encoded = df_encoded.drop(columns=["bairro"])

# Passo 3: Adicionar as colunas codificadas ao DataFrame original
df_encoded = pd.concat([df_encoded, bairro_dummies], axis=1)

In [ ]:
# Vamos exibir todas as colunas que começam com 'bairro_' para ver como ficou a codificação
print(f"Número de Colunas: {len(df_encoded.filter(like='bairro_').columns)}")
print(f"Colunas: {df_encoded.filter(like='bairro_').columns}")

In [ ]:
# Contar quantas colunas foram geradas no processo de One-Hot Encoding
num_total_columns_before = df_encoded.shape[1]
num_total_columns_after = df_encoded.shape[1]

# Exibir informações detalhadas sobre o processo de codificação
print(f"Número total de colunas antes da codificação (df): {num_total_columns_before}")
print(
    f"Número total de colunas após a codificação (df_encoded): {num_total_columns_after}"
)

##### Encoding 'perfil_consumo'


Relembrando as categorias de 'perfil_consumo'


In [ ]:
feature = "perfil_consumo"
unique_values = df_encoded[feature].unique()
cardinality = sorted_cardinalities[feature]

print(f"Feature: {feature}")
print(f"Cardinalidade: {cardinality}")
print(f"Primeiros 10 Valores Únicos: {unique_values[:10]}\n")

In [ ]:
# Passo 1: Criar as colunas binárias para cada perfil de consumo
df_encoded["perfil_aquecedor"] = df_encoded["perfil_consumo"].apply(
    lambda x: 1 if "Aquecedor" in str(x) else 0
)
df_encoded["perfil_coccao"] = df_encoded["perfil_consumo"].apply(
    lambda x: 1 if "Cocção" in str(x) else 0
)
df_encoded["perfil_piscina"] = df_encoded["perfil_consumo"].apply(
    lambda x: 1 if "Piscina" in str(x) else 0
)

# Passo 2: Verificar o resultado
print(
    df_encoded[
        ["perfil_consumo", "perfil_aquecedor", "perfil_coccao", "perfil_piscina"]
    ].sample(20)
)

# Passo 3: Remover a coluna original 'perfil_consumo'
df_encoded = df_encoded.drop(columns=["perfil_consumo"])

Nota: não precisamos alterar os valores 'nan' ou '-', vistos nos valores únicos printados acima, porque esses valores serão codificados como 'False' nas 3 colunas binárias criadas para 'perfil_consumo'.


In [ ]:
# Calcular os percentuais de True (1) em cada coluna de perfil
percentuais_perfil = (
    df_encoded[["perfil_aquecedor", "perfil_coccao", "perfil_piscina"]].mean() * 100
)

# Exibir os percentuais
print("Percentuais de True (1) em cada coluna de perfil:")
print(percentuais_perfil)

# Filtrar os casos onde todos os 3 perfis são False (0)
casos_sem_perfil = df_encoded[
    (df_encoded["perfil_aquecedor"] == 0)
    & (df_encoded["perfil_coccao"] == 0)
    & (df_encoded["perfil_piscina"] == 0)
]

# Calcular e exibir o percentual de casos onde todos os perfis são False
percentual_sem_perfil = (casos_sem_perfil.shape[0] / len(df_encoded)) * 100
print(
    f"\nPercentual de casos onde todos os perfis são False (0): {percentual_sem_perfil:.2f}%"
)

### Variáveis Numéricas


#### Análise


In [ ]:
df_encoded[numerical_features].describe().apply(
    lambda x: x.apply("{:.2f}".format)
)  # Mostra as estatisticas descritivas das colunas numéricas

In [ ]:
# Obtenha as estatísticas descritivas
stats = df_encoded[numerical_features].describe()

# Calcule a razão do desvio-padrão sobre a média para cada coluna numérica
std_dev_to_mean_ratio = stats.loc["std"] / stats.loc["mean"]

print(std_dev_to_mean_ratio)

Podemos notar que das variáveis numéricas iniciais (antes da codificação das variáveis categóricas) a 'pulseCount' se sobressai pelo seu alto desvio-padrão, por isso aplicaremos uma transformação logarítmica.


#### Codificação


##### Transformação logarítmica da coluna pulseCount


In [ ]:
# Aplica a transformação logarítmica na coluna pulseCount e salvar no df_encoded
df_encoded["log_pulseCount"] = np.log1p(
    df_encoded["pulseCount"]
)  # Aplica a transformação logarítmica na coluna pulseCount

# Verifica algumas linhas aleatórias para garantir que a transformação foi bem-sucedida
print(df_encoded[["pulseCount", "log_pulseCount"]].sample(10))

##### Normalização da Variável Gain


In [189]:
# Normalização da variável gain
df_encoded["gain"] = (df_encoded["gain"] - df_encoded["gain"].mean()) / df_encoded[
    "gain"
].std()

#### Entendimento


In [ ]:
# Obtém os tipos de dados das colunas
dtypes = df_encoded.dtypes

# Cria listas de colunas com base no dtype
object_cols = [col for col in dtypes.index if dtypes[col] == "object"]
non_object_cols = [col for col in dtypes.index if dtypes[col] != "object"]

# Cria a lista de colunas ordenada
sorted_cols = object_cols + non_object_cols

# Reordena o DataFrame conforme a lista ordenada
df_sorted = df_encoded[sorted_cols]

print("Informações do dataframe codificado (df_encoded) ordenado pelo data type:\n")
print(df_sorted.info())

Podemos notar que as únicas colunas que não são númericas são as colunas identificadores ('id_features'), assim nossa codificação está completa!

Nota: poderíamos utilizar um escalador (como o MinMaxScaler do Sklearn) nas colunas a partir de agora, mas escolhemos preservar a escala dos dados para manter a interpretabilidade e evitar limitações para a próxima etapa de feature engineering e criação de modelos.


In [ ]:
print(id_features)

### Tratamento de Outliers


Neste projeto, os outliers não serão removidos, pois o foco é identificar anomalias no consumo de gás. Em vez disso, estamos tratando os outliers como casos de interesse que precisam ser analisados e modelados adequadamente. A seguir, são apresentadas as técnicas utilizadas para a detecção e análise desses outliers.


##### Visualização de Outliers


In [ ]:
# Visualização dos outliers em meterIndex e pulseCount
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[['meterIndex', 'pulseCount']])
plt.title('Visualização dos Outliers em meterIndex e pulseCount')
plt.show()

## 4.3. Preparação dos Dados e Modelagem

Nesta seção, será criada a primeira versão do modelo preditivo, as escolhas de features e resultados serão explicados e analisados.

### 4.3.1. Modelo Não-Supervisionado - Modelagem, features e explicação

_Hipotese inicial_
Partimos da premissa de que dispúnhamos de uma quantidade suficiente de CEPs para realizar comparações significativas. Acreditávamos que indivíduos que moram próximos uns dos outros apresentariam padrões de consumo semelhantes. Nossa abordagem consistia em avaliar a variação da taxa de consumo ao longo de uma semana e, em seguida, comparar esses dados com indivíduos agrupados em clusters específicos.

No entanto, enfrentamos um problema: a quantidade de CEPs disponível não foi suficiente para realizar uma análise robusta. Como resultado, não conseguimos formar clusters que fossem suficientemente representativos para validar nossa hipótese.

_Nova Abordagem:_

Dado que decidimos não seguir com a hipótese inicial, adotamos uma nova estratégia para análise. Nossa abordagem agora foca na comparação entre o consumo individual e o comportamento médio/mediano da residência por dia da semana. A seguir, detalhamos os principais passos e considerações:

Análise Residência vs. Indivíduo por Dia da Semana:

O consumo de cada indivíduo será comparado com a média ou mediana de consumo do prédio (condomínio) onde reside.
A partir dessa comparação, os resultados entre os indivíduos serão agregados e comparados com a média ou mediana do consumo de cada agrupamento (condIndex).
A renda per capita, inferida pelo condCode, também será considerada como um fator influente.
Cada grupo de indivíduos dentro de um mesmo condomínio (condCode) será tratado como um cluster. No entanto, é importante notar que nem todos os consumidores estão vinculados a um condomínio.

Justificativa da Escolha das Features:

pulseCount (Contagem de Pulsos):
Esta feature é fundamental, pois representa a base do cálculo de consumo de gás. Cada pulso captado pelo medidor está diretamente relacionado à quantidade de gás consumida. É essencial para medir o consumo de cada cliente de forma precisa e granular.

gain (Fator de Multiplicação de Pulsos):
O fator de multiplicação de pulsos é necessário para converter a contagem de pulsos em uma unidade mais prática, como metros cúbicos de gás. Essa feature garante que os dados de consumo sejam interpretados corretamente, considerando possíveis diferenças entre medidores.

condCode (Código do Condomínio):
Essa feature permite identificar e agrupar consumidores que residem em um mesmo condomínio. A análise por agrupamentos, como clusters de consumidores que compartilham o mesmo condCode, facilita a identificação de padrões coletivos e anomalias que podem ser específicas de um determinado condomínio.

clientIndex (Identificador Único):
O clientIndex é essencial para diferenciar os consumidores individuais. Ele garante que cada unidade de medição seja tratada de forma única, permitindo análises individualizadas mesmo dentro de agrupamentos maiores, como os de condomínio.

_pulseCount será multiplicado pelo Gain_

Análise Comparativa:

Pontos Positivos:
Possibilita a identificação de padrões de consumo anômalos com base no comportamento diário.
Facilita a segmentação dos consumidores em clusters específicos, como os agrupados por condomínio, com base no consumo diário.
Pontos Negativos:
Medições coletivas podem influenciar os dados, dificultando a análise individual.
A limitação de informações específicas por condomínio pode restringir a precisão da formação dos clusters.


_Início do desenvolvimento da abordagem:_


Criação da coluna totalConsumption


In [193]:
df["totalConsumption"] = df.pulseCount * df.gain

Abaixo faremos a validação da hipotese de se é proveitoso ou não agrupar os clusters por condomínio.


Contagem de quem mais aparece na coluna condCode.


In [ ]:
condCodeValues = df["condCode"].value_counts()
condCodeValues

Pegando o condomínio que mais aparece.


In [ ]:
targetCondCode = condCodeValues.index[0]
print(targetCondCode)

Criando o targetCondCodeDf, contendo valores somente desse condomínio que mais aparece.


In [ ]:
targetCondCodeDf = df[df["condCode"] == targetCondCode]
targetCondCodeDf

_Criando a coluna measure diff:_

A coluna foi criada organizando o df em ordem crescente, primeiro pelo clientCode depois pelo datetime. depois foi criada uma coluna auxiliar, que depois foi removida, chamada 'match', com valores True se a linha n-1 for do mesmo cliente que a linha n. Se a coluna match for true, a coluna measure_diff calcula a diferença de total consumption da linha n para a linha n-1.


In [ ]:
targetCondCodeDf = targetCondCodeDf.sort_values(
    by=["clientCode", "datetime"], ascending=True
)
targetCondCodeDf["match"] = df.clientCode.eq(df.clientCode.shift())
targetCondCodeDf["measure_diff"] = targetCondCodeDf["match"].where(
    targetCondCodeDf["match"] == False, targetCondCodeDf["totalConsumption"].diff()
)
targetCondCodeDf["measure_diff"] = pd.to_numeric(
    targetCondCodeDf["measure_diff"], errors="coerce"
)
targetCondCodeDf = targetCondCodeDf.dropna(subset=["match"])
targetCondCodeDf = targetCondCodeDf.drop(columns=["match"])
targetCondCodeDf.reset_index(drop=True, inplace=True)
targetCondCodeDf

Criação da coluna measure_avg_consumption agrupando os dados por dia e obtendo a média de measure_diff daquele período.


In [198]:
targetCondCodeDf.reset_index(drop=True, inplace=True)

targetCondCodeDf["datetime_yyyy-mm-dd"] = targetCondCodeDf["datetime"].dt.normalize()

targetCondCodeDf["measure_avg_consumption"] = None

targetCondCodeDf = targetCondCodeDf[targetCondCodeDf["measure_diff"] > 0]

for date in np.array(targetCondCodeDf["datetime_yyyy-mm-dd"].unique()):
    measure_avg_consumption = targetCondCodeDf[
        targetCondCodeDf["datetime_yyyy-mm-dd"].dt.to_period("D")
        == pd.to_datetime(date).to_period("D")
    ]
    measure_avg_consumption = measure_avg_consumption["measure_diff"].mean()
    targetCondCodeDf.loc[
        targetCondCodeDf["datetime_yyyy-mm-dd"] == date, "measure_avg_consumption"
    ] = measure_avg_consumption

Pegando os dados de um mês aleatório.


In [199]:
filtered_df = targetCondCodeDf[
    targetCondCodeDf["datetime"].dt.to_period("M")
    == pd.to_datetime("2024-06").to_period("M")
]

Gráfico de média de consumo do condomínio por dia vs usuários.


In [ ]:
plt.figure(figsize=(25, 6))

plt.scatter(
    filtered_df["datetime"],
    filtered_df["measure_diff"],
    label="Measure Consumption",
    color="blue",
)

plt.scatter(
    filtered_df["datetime"],
    filtered_df["measure_avg_consumption"],
    label="Average Consumption",
    color="orange",
)

plt.title("Total vs Average Consumption Over Time")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=45)

plt.legend()
plt.tight_layout()
plt.show()

Pegando um clientCode aleatório dentro aquele condomínio.


In [ ]:
target_clientCode = targetCondCodeDf["clientCode"].value_counts().index[1]
print(target_clientCode)

Gráfico do consumo desse cliente vs a média do condomínio.


In [ ]:
plt.figure(figsize=(25, 6))

plt.plot(
    filtered_df[filtered_df["clientCode"] == target_clientCode]["datetime"],
    filtered_df[filtered_df["clientCode"] == target_clientCode]["measure_diff"],
    label="Measure Consumption",
    color="blue",
)

plt.plot(
    filtered_df[filtered_df["clientCode"] == target_clientCode]["datetime"],
    filtered_df[filtered_df["clientCode"] == target_clientCode][
        "measure_avg_consumption"
    ],
    label="Average Consumption",
    color="orange",
)

plt.title("Total vs Average Consumption Over Time")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=45)

plt.legend()
# plt.ylim(0, 0.6)
plt.tight_layout()
plt.show()

Agora, aplicando a lógica de somente um condomínio de measure_diff para todo o df.


In [ ]:
df["match"] = df.clientCode.eq(df.clientCode.shift())
df["measure_diff"] = df["match"].where(
    df["match"] == False, df["totalConsumption"].diff()
)
df["measure_diff"] = pd.to_numeric(df["measure_diff"], errors="coerce")
df = df.dropna(subset=["match"])
df = df.drop(columns=["match"])
df.reset_index(drop=True, inplace=True)
df

Agora, aplicando a lógica de somente um condomínio de measure_avg_consumption para todo o df.


In [204]:
df.reset_index(drop=True, inplace=True)

df["datetime_yyyy-mm-dd"] = df["datetime"].dt.normalize()

df["measure_avg_consumption"] = None

for date in np.array(df["datetime_yyyy-mm-dd"].unique()):
    measure_avg_consumption = df[
        df["datetime_yyyy-mm-dd"].dt.to_period("D")
        == pd.to_datetime(date).to_period("D")
    ]
    measure_avg_consumption = measure_avg_consumption["measure_diff"].mean()
    df.loc[df["datetime_yyyy-mm-dd"] == date, "measure_avg_consumption"] = (
        measure_avg_consumption
    )

Escolha das colunas numéricas.


In [205]:
df = df[
    ["clientCode", "condCode", "timestamp", "measure_avg_consumption", "measure_diff"]
]

Label enconding de condCode e clientCode.


In [ ]:
from sklearn.preprocessing import LabelEncoder

le_condCode = LabelEncoder()
le_clientCode = LabelEncoder()


le_condCode.fit(df["condCode"].dropna())
df["C_condCode"] = pd.Series(
    le_condCode.transform(df["condCode"].dropna()), index=df["condCode"].dropna().index
)
df["C_condCode"] = df["C_condCode"].reindex(df.index).astype(float)


le_clientCode.fit(df["clientCode"].dropna())
df["C_clientCode"] = pd.Series(
    le_clientCode.transform(df["clientCode"].dropna()),
    index=df["clientCode"].dropna().index,
)
df["C_clientCode"] = df["C_clientCode"].reindex(df.index).astype(float)

Gráfico da média de consumo de todos os condiminios por dia vs consumo de todos usuários.


In [ ]:
plt.figure(figsize=(25, 15))

plt.scatter(
    df["timestamp"],
    df["measure_diff"],
    label="Measure Consumption",
    color="blue",
)

plt.scatter(
    df["timestamp"],
    df["measure_avg_consumption"],
    label="Average Consumption",
    color="orange",
)

plt.title("Total vs Average Consumption Over Time")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=45)

plt.legend()
plt.ylim(0, 20)
plt.tight_layout()
plt.show()

### 4.3.2. Modelo Candidato

O modelo escolhido é o **KMeans**, um algoritmo de clustering amplamente utilizado para agrupar dados com base em similaridades. Neste caso, foram selecionadas três features relevantes:

- **measure_diff**: Diferença de medidas de consumo de gás.
- **C_condCode**: Código do condomínio para identificar agrupamentos.
- **C_clientCode**: Código do cliente, que representa cada unidade de medição individual.

O modelo foi configurado para gerar **3 clusters** e distribuiu os indivíduos com base na similaridade dessas variáveis. Os clusters foram visualizados em um gráfico 3D, com os centróides claramente marcados para indicar os pontos centrais de cada grupo.

### Discussão sobre os Resultados

O KMeans conseguiu segmentar os consumidores de maneira eficaz, agrupando-os em três clusters distintos. Cada cluster representa um padrão comum de consumo de gás, levando em consideração a localização (condCode) e o comportamento individual (clientCode).

- **Os clusters formados** revelam agrupamentos de consumidores com padrões de consumo semelhantes. Isso facilita a análise de comportamento coletivo (dentro de condomínios) e individual.
- **Os centróides indicam** os padrões médios de consumo de cada grupo, sendo pontos de referência claros para comparar consumidores dentro do cluster.

**Conclusão:**  
A escolha do KMeans com 3 clusters forneceu uma segmentação clara e útil dos consumidores, destacando padrões de consumo distintos e criando uma base sólida para análise de comportamento. Essa segmentação permite identificar anomalias ou padrões específicos que podem ser explorados.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

df = df.dropna()

X = df[["measure_diff", "C_condCode", "C_clientCode"]]

kmeans = KMeans(n_clusters=3, random_state=42)
df["cluster"] = kmeans.fit_predict(X)

# Configurar o gráfico em 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection="3d")

# Plotar os dados, usando as três features e colorindo de acordo com os clusters
scatter = ax.scatter(
    df["measure_diff"],
    df["C_condCode"],
    df["C_clientCode"],
    c=df["cluster"],
    cmap="viridis",
    s=50,
)

# Adicionar os centróides no gráfico
centroids = kmeans.cluster_centers_
ax.scatter(
    centroids[:, 0],
    centroids[:, 1],
    centroids[:, 2],
    s=300,
    c="red",
    label="Centróides",
)

# Configurar rótulos e título
ax.set_title("Clusters 3D - KMeans")
ax.set_xlabel("Measure_diff")
ax.set_ylabel("C_condCode")
ax.set_zlabel("C_clientCode")

# Adicionar uma legenda
legend = ax.legend(loc="upper right", title="Centróides")

# Mostrar o gráfico
plt.show()

### 4.3.3. Definição do K e Justificativa

Inicialmente, definimos o valor de **K = 3** de forma arbitrária para segmentar os consumidores. No entanto, para garantir uma escolha mais embasada, utilizamos duas métricas:

1. **Elbow Method**: Essa métrica sugere o K ideal ao avaliar a variação na soma das distâncias dentro dos clusters. O gráfico do método Elbow indicou que o valor ideal de K estaria entre **4 e 5**, já que a curva começou a suavizar nesse ponto.

2. **Silhouette Score**: Essa métrica mede a coesão e separação dos clusters. O Silhouette Score recomendou o valor de **K = 3**, indicando que, com 3 clusters, os grupos estavam mais bem definidos, com maior distância entre clusters e melhor compactação interna.

Seguindo a recomendação do **Silhouette Score**, optamos por manter **K = 3**, pois essa métrica demonstrou que esse número de clusters proporcionava uma segmentação mais clara e precisa dos consumidores.


### Definição do Elbow.


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features = df[["measure_diff", "C_condCode", "C_clientCode"]]

features = features.dropna()

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

k_values = range(1, 11)
inertia = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, "bo-", markersize=8)
plt.xlabel("Número de clusters (K)")
plt.ylabel("Inércia")
plt.title("Método Elbow para encontrar o melhor K")
plt.grid(True)
plt.show()

### Gráfico da Silhueta


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Usando 10% dos dados como no exemplo anterior
df_sample = df.sample(frac=0.1, random_state=42)
X = df_sample[["measure_diff", "C_condCode", "C_clientCode"]]
X = X.dropna()

# Defina o valor de K (número de clusters que você deseja usar)
n_clusters = 3  # Exemplo, escolha o K apropriado

# Criando o modelo KMeans e ajustando aos dados
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X)

# Calculando o coeficiente da silhueta para cada ponto
silhouette_avg = silhouette_score(X, cluster_labels)
print(f"Coeficiente médio de silhueta para K = {n_clusters}: {silhouette_avg}")

# Calcula os valores da silhueta para cada amostra
sample_silhouette_values = silhouette_samples(X, cluster_labels)

# Iniciando o gráfico
fig, ax1 = plt.subplots(1, 1)
fig.set_size_inches(10, 7)

# Configuração do gráfico
ax1.set_xlim([-0.1, 1])  # O coeficiente da silhueta vai de -1 a 1
ax1.set_ylim(
    [0, len(X) + (n_clusters + 1) * 10]
)  # Para deixar espaço entre os clusters

y_lower = 10
for i in range(n_clusters):
    # Agrega os valores da silhueta para o cluster i
    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / n_clusters)
    ax1.fill_betweenx(
        np.arange(y_lower, y_upper),
        0,
        ith_cluster_silhouette_values,
        facecolor=color,
        edgecolor=color,
        alpha=0.7,
    )

    # Rotula os clusters no meio do gráfico
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Calcula a nova posição do eixo y
    y_lower = y_upper + 10  # 10 para o espaço entre os clusters

# Adicionar títulos e labels ao gráfico
ax1.set_title("Gráfico de Silhueta para os clusters")
ax1.set_xlabel("Coeficiente de Silhueta")
ax1.set_ylabel("Cluster")

# A linha vertical para a média do coeficiente de silhueta
ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

# Remover labels y
ax1.set_yticks([])

# Exibir o gráfico
plt.show()

### 4.3.4. Escolha do Tipo de Sistema de Recomendação

Optamos por um **sistema de recomendação baseado em análise de clusters e detecção de anomalias**, adequado para o cenário de monitoramento de consumo de gás. Essa abordagem permite que o sistema:

- **Identifique padrões de consumo anômalos** que desviam do comportamento usual de um determinado cluster, o que pode indicar vazamentos ou falhas em equipamentos.
- **Sugira áreas prioritárias para intervenção técnica**, utilizando dados de proximidade geográfica e consumo para identificar regiões que necessitam de manutenção preventiva.
- **Compare o consumo individual com a média do grupo**, facilitando a detecção de discrepâncias que justifiquem inspeções ou ajustes, aumentando a eficiência no controle e gestão do consumo de gás.

Essa escolha é justificada pela necessidade de um sistema que detecte variações no comportamento de consumo e priorize ações preventivas de forma precisa e ágil.


### Abaixo segue análise de cliente que mais aparece com medidas acima de 20.

Exemplo de como os insights do modelo devem ser utilizados.


In [ ]:
df[df["measure_diff"] > 20]["C_clientCode"].value_counts()

DF desse cliente.


In [ ]:
df[df["C_clientCode"] == 1685]

Coluna datetime a partir da timestamp.


In [213]:
import datetime

df["datetime"] = df["timestamp"].apply(
    lambda x: (datetime.datetime.fromtimestamp(x) if pd.notnull(x) else None)
)

Usuário vs média de seu condominio em diferentes escalas.

In [ ]:
plt.figure(figsize=(25, 15))

plt.scatter(
    df[(df["C_clientCode"] == 1685)]["datetime"],
    df[(df["C_clientCode"] == 1685)]["measure_diff"],
    label="Consumo do outlier",
    color="blue",
)

plt.scatter(
    df[(df["C_condCode"] == 36)]["datetime"].sort_values(ascending=True),
    df[(df["C_condCode"] == 36)]["measure_avg_consumption"],
    label="Consumo médio do condomínio",
    color="orange",
)

plt.title("Consumo do outlier vs Consumo do condomínio")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=90)

plt.legend()
plt.tight_layout()
plt.show()

# Usuário vs Média de Consumo no Condomínio em Diferentes Escalas

Neste gráfico, estamos comparando o consumo de um cliente específico (código 1685) com o consumo médio de um condomínio (código 36) ao longo do tempo. 

- **Pontos azuis**: Representam o consumo de um "outlier", ou seja, um cliente cujo consumo se distancia significativamente do comportamento esperado.
- **Pontos laranja**: Representam o consumo médio de todos os usuários do condomínio 36 no mesmo período.

Essa visualização é importante para identificarmos padrões anômalos em comparação com o consumo regular do condomínio. O comportamento do cliente fora da curva pode sugerir a necessidade de uma investigação mais detalhada, como possíveis fraudes, falhas no medidor ou vazamentos.

A análise do gráfico mostra que o consumo do outlier tem um comportamento muito mais volátil e disperso do que a média estável do condomínio.


In [ ]:
plt.figure(figsize=(25, 15))

plt.scatter(
    df[(df["C_clientCode"] == 1685) & (df["datetime"] > "2024-05")]["datetime"],
    df[(df["C_clientCode"] == 1685) & (df["datetime"] > "2024-05")]["measure_diff"],
    label="Consumo do outlier",
    color="blue",
)

plt.scatter(
    df[(df["C_condCode"] == 36) & (df["datetime"] > "2024-05")]["datetime"],
    df[(df["C_condCode"] == 36) & (df["datetime"] > "2024-05")][
        "measure_avg_consumption"
    ],
    label="Consumo médio do condomínio",
    color="orange",
)

plt.title("Consumo do outlier vs Consumo do condomínio")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=90)

plt.legend()
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

# Análise Refinada do Período após Maio de 2024

Neste gráfico, restringimos a análise para datas posteriores a maio de 2024, focando em um intervalo mais recente de consumo. 

- **Escala Ajustada**: A escala do eixo Y foi ajustada para entre 0 e 1 para melhorar a visualização das diferenças no consumo médio e do outlier nesse período.
- **Padrões Visíveis**: Embora o consumo médio continue estável, o consumo do outlier exibe picos que devem ser observados de perto.

A escolha de um período mais curto nos permite visualizar com maior clareza qualquer variação anômala no comportamento de consumo do cliente, facilitando a identificação de eventos significativos para análises mais detalhadas.


In [ ]:
plt.figure(figsize=(25, 15))

plt.scatter(
    df[(df["C_clientCode"] == 1685) & (df["datetime"] > "2024-05")]["datetime"],
    df[(df["C_clientCode"] == 1685) & (df["datetime"] > "2024-05")]["measure_diff"],
    label="Consumo do outlier",
    color="blue",
)

plt.scatter(
    df[(df["C_condCode"] == 36) & (df["datetime"] > "2024-05")]["datetime"],
    df[(df["C_condCode"] == 36) & (df["datetime"] > "2024-05")][
        "measure_avg_consumption"
    ],
    label="Consumo médio do condomínio",
    color="orange",
)

plt.title("Consumo do outlier vs Consumo do condomínio")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=90)

plt.legend()
plt.tight_layout()
plt.show()

Para cálculo do consumo de cada consumidor, precisamos que o consumo leve em conta a ausência de acrescimo do pulseCount apesar do aumento do consumo. A ausência dessa relação incorre na falsa impressão de que um consumidor teve um alto consumo repentino, gerando spikes de consumo e dando uma falsa indicação
Pegar o consumo e dividir entre o consumo médio acumulado entre medições

Dividir o consumo médio pelo delta do tempo


# Sprint 4: Competição entre modelos
#### Abaixo desenvolveremos 3 modelos a partir de hipóteses e avaliações diferentes que serão explicados abaixo

## Modelo 1

# Cálculo do Consumo Médio e Detecção de Picos

Nesta etapa, o objetivo é calcular o consumo de cada cliente levando em consideração a ausência de aumento no contador de pulsos (`pulseCount`), o que pode resultar em picos falsos de consumo. Para evitar essas inconsistências, o consumo médio entre medições é utilizado, dividido pelo delta de tempo entre cada leitura.

**Etapas:**
1. **Ordenação dos Dados**: O dataframe é ordenado pelo código do cliente (`C_clientCode`) e pela data da medição (`datetime`), para garantir que as comparações entre leituras consecutivas façam sentido.
2. **Criação da Coluna `match`**: A nova coluna `match` é criada para identificar se a linha atual pertence ao mesmo cliente que a linha anterior, facilitando a detecção de mudanças no cliente ou no medidor.

Essa abordagem ajuda a evitar a detecção de falsos picos de consumo que podem ocorrer devido à falta de incremento do contador de pulsos, garantindo uma análise mais robusta dos dados.

In [217]:
df_1 = df.copy()

In [ ]:
df_1 = df_1.sort_values(by=["C_clientCode", "datetime"], ascending=True)

df_1["match"] = df_1.clientCode.eq(df_1.clientCode.shift())

df_1

In [ ]:
df_1.reset_index(drop=True, inplace=True)
df_1

In [ ]:
x = df_1["datetime"][1] - df_1["datetime"][0]
print(x.total_seconds())

In [221]:
from datetime import timedelta

df_1["time_diff_secs"] = None

df_1["time_diff_secs"] = df_1["match"].where(df_1["match"] == False, df_1["datetime"].diff())

df_1["time_diff_secs"] = df_1["time_diff_secs"].where(
    df_1["match"] != False,
    timedelta(seconds=0).total_seconds(),
)

In [ ]:
df_1

# Cálculo do Delta de Tempo Entre Medições

Nesta etapa, estamos calculando a diferença de tempo (`time_diff_secs`) entre medições consecutivas para cada cliente, o que é fundamental para calcular o consumo corretamente ao longo do tempo.

**Etapas:**
1. **Reset do Índice**: O índice do dataframe foi redefinido para garantir que as operações subsequentes sejam realizadas de forma sequencial após a ordenação anterior.
2. **Cálculo da Diferença de Tempo**: A diferença de tempo é calculada usando a função `.diff()` para determinar o intervalo entre cada medição.
   - Caso seja o mesmo cliente, o delta de tempo real é calculado.
   - Caso o cliente mude entre as medições (valor `False` na coluna `match`), o delta de tempo é ajustado para `0` segundos, evitando cálculos incorretos.

Este cálculo de delta de tempo é importante para garantir que as comparações de consumo sejam consistentes entre medições e que qualquer falha na continuidade da medição seja devidamente tratada.


In [223]:
df_1["time_diff_secs"] = df_1["time_diff_secs"].apply(
    lambda x: x.total_seconds() if x != 0.0 else x
)

In [ ]:
df_1

In [ ]:
df_1[df_1["C_clientCode"] == 1716.0]

In [ ]:
df_1["measure_by_time"] = df_1["measure_diff"] / df_1["time_diff_secs"]
df_1

# Ajuste do Delta de Tempo e Cálculo do Consumo por Unidade de Tempo

Nesta etapa, estamos refinando o cálculo do delta de tempo e utilizando essa métrica para calcular o consumo por unidade de tempo para cada cliente.

**Etapas:**
1. **Ajuste do Delta de Tempo**: O delta de tempo em segundos foi refinado utilizando a função `apply()` com uma `lambda function`. Isso garante que os casos onde o delta de tempo é zero sejam tratados adequadamente. Se o delta for zero, mantemos o valor de zero, caso contrário, convertemos para o total em segundos.
2. **Filtragem por Cliente**: Filtramos os dados para focar no cliente de código `1716.0`, permitindo uma análise mais detalhada de um único cliente para verificar seus padrões de consumo.
3. **Cálculo do Consumo por Tempo (`measure_by_time`)**: O consumo diferenciado (`measure_diff`) é dividido pelo delta de tempo em segundos para obter o consumo por unidade de tempo. Essa métrica é essencial para padronizar o consumo e identificar padrões de comportamento ao longo do tempo.

Essas operações ajudam a garantir que o consumo seja calculado de forma precisa, mesmo em intervalos de tempo irregulares, o que facilita a detecção de anomalias no comportamento de consumo.


In [ ]:
df_1["measure_by_time"] = df_1["measure_by_time"].replace(np.inf, 0)
df_1["measure_by_time"] = df_1["measure_by_time"].replace(np.nan, 0)
df_1.info()

# Normalização dos Dados e Clustering com K-Means

Nesta etapa, estamos preparando os dados para a aplicação do algoritmo de clustering K-Means. O objetivo é identificar grupos (clusters) de comportamento de consumo similares entre os clientes.

**Etapas:**
1. **Tratamento de Valores Infinitos e Nulos**: 
   - Primeiramente, os valores infinitos e `NaN` na coluna `measure_by_time` são substituídos por zero, garantindo que o dataset não contenha valores inválidos que possam comprometer o algoritmo.
   - Em seguida, removemos qualquer linha que contenha valores nulos nas colunas de features selecionadas.

2. **Normalização dos Dados**: 
   - Usamos o `StandardScaler` para normalizar os dados, o que é crucial para algoritmos como o K-Means, já que ele é sensível à escala das features.

3. **Aplicação do K-Means**: 
   - Testamos diferentes valores de `K` (número de clusters) para o K-Means, variando de 1 a 10 clusters.
   - Para cada valor de `K`, calculamos a inércia, que é a soma das distâncias quadradas dentro de cada cluster.

4. **Plot do Método Elbow**: 
   - O gráfico "Elbow" (cotovelo) é gerado para visualizar a inércia em função do número de clusters. O ponto onde o gráfico faz uma "curva" é sugerido como o valor ideal de `K`.
   - Isso ajuda a determinar o número de clusters que melhor segmenta os padrões de consumo dos clientes.

Essa análise nos permite agrupar clientes com padrões de consumo similares, facilitando a identificação de anomalias ou comportamentos fora do padrão em cada cluster.


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features = df_1[["measure_by_time", "C_condCode", "C_clientCode"]]
features.replace([np.inf, -np.inf], np.nan, inplace=True)

features = features.dropna()

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

k_values = range(1, 11)
inertia = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, "bo-", markersize=8)
plt.xlabel("Número de clusters (K)")
plt.ylabel("Inércia")
plt.title("Método Elbow para encontrar o melhor K")
plt.grid(True)
plt.show()

# Refinamento do Consumo por Tempo e Análise por Data

**Gráfico Elbow**: O gráfico acima mostra o resultado da aplicação do **método Elbow** para identificar o número ideal de clusters. Aqui, podemos observar que a inércia diminui drasticamente até cerca de 3-4 clusters, o que sugere que esse pode ser o número ideal de clusters para segmentar os dados.

Após essa análise, continuamos refinando o dataset:

**Etapas:**
1. **Tratamento de Valores Infinitos e Nulos**: Assim como nas etapas anteriores, os valores infinitos (`np.inf`) e `NaN` são removidos para garantir a integridade dos dados.
   
2. **Escalonamento do Consumo**: 
   - A coluna `measure_by_time` é escalada por um fator de 100, armazenada na nova coluna `measure_by_time_scaled`, para normalizar os valores de consumo e facilitar comparações subsequentes.
   - Filtramos para manter apenas os registros onde o consumo escalado é maior que zero, garantindo que apenas dados válidos sejam considerados.

3. **Análise por Data e Condomínio**: 
   - Normalizamos a data (`datetime`) para criar uma coluna no formato `YYYY-MM-DD`, facilitando a agregação por dias.
   - Para cada data única e cada condomínio, calculamos o consumo médio diário (`measure_by_time_avg_consumption`), garantindo que tenhamos uma métrica confiável de consumo para cada dia e condomínio.

Este refinamento é essencial para garantir a consistência dos dados ao longo do tempo e facilitar a análise de padrões diários de consumo, permitindo que anomalias possam ser detectadas em diferentes escalas temporais.


In [229]:
df_1["measure_by_time"] = df_1["measure_by_time"].replace([np.inf, -np.inf], np.nan)
df_1["measure_by_time"] = df_1["measure_by_time"].dropna()

df_1["measure_by_time_scaled"] = df_1["measure_by_time"] * 100
df_1 = df_1[df_1["measure_by_time_scaled"] > 0]

In [ ]:
df_1.reset_index(inplace=True, drop=True)
df_1["datetime_yyyy-mm-dd"] = df_1["datetime"].dt.normalize()

for date in np.array(df_1["datetime_yyyy-mm-dd"].unique()):
    measure_days_df_1 = df_1[df_1["datetime_yyyy-mm-dd"] == date]
    for cond in np.array(measure_days_df_1["C_condCode"].unique()):
        measure_avg_consumption = measure_days_df_1[
            measure_days_df_1["C_condCode"] == cond
        ]["measure_by_time"].mean()
        df_1.loc[
            ((df_1["datetime_yyyy-mm-dd"] == date) & (df_1["C_condCode"] == cond)),
            "measure_by_time_avg_consumption",
        ] = measure_avg_consumption

In [ ]:
df_1

# Comparação do Consumo Total vs Consumo Médio ao Longo do Tempo

Neste gráfico, estamos comparando duas métricas importantes de consumo ao longo do tempo:

1. **Consumo Total (Medido)**:
   - Representado pelos **pontos azuis**, essa métrica (`measure_by_time`) indica o consumo total medido para cada ponto no tempo.
   
2. **Consumo Médio**:
   - Representado pelos **pontos laranja**, essa métrica (`measure_by_time_avg_consumption`) reflete o consumo médio diário do condomínio. É útil para observar a tendência geral de consumo ao longo do tempo.

**Objetivo do Gráfico**:
- Visualizar como o consumo total se comporta em relação à média.
- Identificar períodos onde o consumo total se desvia da média, o que pode indicar anomalias, picos de consumo ou eventos específicos que devem ser investigados.

**Configurações do Gráfico**:
- O eixo Y está limitado entre 0 e 0.01 para facilitar a visualização de pequenas variações de consumo ao longo do tempo.
- A rotação dos rótulos de data no eixo X foi ajustada para melhorar a legibilidade das datas ao longo do tempo.

Esse gráfico é útil para identificar possíveis outliers, picos de consumo e avaliar a eficiência energética ao comparar o comportamento real com a média.


In [ ]:
plt.figure(figsize=(25, 15))

plt.scatter(
    df_1["datetime"],
    df_1["measure_by_time"],
    label="Measure Consumption",
    color="blue",
)

plt.scatter(
    df_1["datetime"],
    df_1["measure_by_time_avg_consumption"],
    label="Average Consumption",
    color="orange",
)

plt.title("Total vs Average Consumption Over Time")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=45)

plt.legend()
plt.ylim(0, 0.01)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

# Selecionar as features para o KMeans
X = df_1[["measure_by_time", "C_condCode", "C_clientCode"]]

# Aplicar o KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
df_1["cluster"] = kmeans.fit_predict(X)

# Configurar o gráfico em 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection="3d")

# Plotar os dados, usando as três features e colorindo de acordo com os clusters
scatter = ax.scatter(
    df_1["measure_by_time"],
    df_1["C_condCode"],
    df_1["C_clientCode"],
    c=df_1["cluster"],
    cmap="viridis",
    s=50,
)

# Adicionar os centróides no gráfico
centroids = kmeans.cluster_centers_
ax.scatter(
    centroids[:, 0],
    centroids[:, 1],
    centroids[:, 2],
    s=300,
    c="red",
    label="Centróides",
)

# Configurar rótulos e título
ax.set_title("Clusters 3D - KMeans")
ax.set_xlabel("Measure_by_time")
ax.set_ylabel("C_condCode")
ax.set_zlabel("C_clientCode")

# Adicionar uma legenda
legend = ax.legend(loc="upper right", title="Centróides")

# Mostrar o gráfico
plt.show()

# Visualização 3D dos Clusters com K-Means

Neste gráfico 3D, utilizamos o algoritmo K-Means para agrupar os dados de consumo com base em três variáveis principais:
- **Measure_by_time**: Consumo por unidade de tempo.
- **C_condCode**: Código do condomínio.
- **C_clientCode**: Código do cliente.

**Etapas:**
1. **Aplicação do K-Means**: 
   - O algoritmo foi ajustado para gerar 3 clusters com base nas features selecionadas.
   - O resultado da previsão (`cluster`) é adicionado ao dataframe para colorir os pontos no gráfico de acordo com os clusters atribuídos.

2. **Visualização em 3D**: 
   - As três dimensões do gráfico representam as variáveis de entrada. Cada ponto é colorido conforme o cluster ao qual pertence, permitindo uma visão clara da segmentação dos dados.
   - Foram adicionados os **centroides** dos clusters em vermelho, que representam o "centro" de cada grupo. Esses pontos ajudam a visualizar onde estão concentradas as características médias de cada cluster.

**Objetivo do Gráfico**:
- A visualização 3D permite explorar a distribuição dos dados e identificar como os clientes e condomínios estão agrupados com base no consumo. 
- Clusters bem definidos podem ajudar a identificar padrões de consumo similares e distinguir outliers ou comportamentos incomuns.

Este tipo de análise é útil para segmentar clientes com comportamentos similares e entender padrões de consumo em diferentes condomínios.


In [ ]:
target_outlier = df_1[df_1["measure_by_time"] > 0.01]["C_clientCode"].value_counts().index[0]
target_outlier

In [ ]:
# plt.figure(figsize=(25, 15))

plt.scatter(
    df_1[df_1["C_clientCode"] == target_outlier]["datetime_yyyy-mm-dd"],
    df_1[df_1["C_clientCode"] == target_outlier]["measure_by_time"],
    label="Measure Consumption",
    color="blue",
)

plt.scatter(
    df_1[df_1["C_clientCode"] == target_outlier]["datetime_yyyy-mm-dd"],
    df_1[df_1["C_clientCode"] == target_outlier]["measure_by_time_avg_consumption"],
    label="Average Consumption",
    color="orange",
)

plt.title("Total vs Average Consumption Over Time")
plt.xlabel("Date")
plt.ylabel("Consumption")


plt.xticks(rotation=45)

plt.legend()
# plt.ylim(0, 0.01)
plt.tight_layout()
plt.show()

# Identificação e Análise de Outlier no Consumo

Nesta etapa, estamos focando em identificar um **outlier** no consumo, ou seja, um cliente cujo consumo é significativamente maior que o padrão observado.

**Etapas:**
1. **Identificação do Outlier**:
   - Usamos um filtro para identificar o cliente cujo `measure_by_time` (consumo por tempo) excede o limite de 0.01, sendo considerado um consumo fora do normal.
   - A variável `target_outlier` armazena o código do cliente identificado como outlier, para que possamos focar em seu comportamento de consumo.

2. **Comparação do Consumo Total com a Média**:
   - No gráfico, o **consumo total** do outlier é representado pelos **pontos azuis**, enquanto o **consumo médio** é mostrado pelos **pontos laranja**.
   - Esta visualização permite observar claramente a discrepância entre o consumo do outlier e a média do condomínio ao longo do tempo.

**Objetivo do Gráfico**:
- Monitorar o comportamento do cliente fora do padrão e verificar se há picos significativos em determinados períodos.
- Comparar o consumo total com a média para entender o quanto esse cliente se distancia do padrão de consumo geral.

Este tipo de análise é crucial para identificar anomalias de consumo que podem indicar problemas como vazamentos, fraudes ou uso atípico de energia.


In [ ]:
df_1[df_1["C_clientCode"] == target_outlier].sort_values(by="datetime")

In [ ]:
df_1[df_1["C_clientCode"] == target_outlier]["datetime_yyyy-mm-dd"].unique()

# Ordenação e Análise Temporal do Outlier

Nesta etapa, estamos examinando mais de perto o comportamento do cliente identificado como outlier. O objetivo é entender melhor o padrão de consumo desse cliente ao longo do tempo.

**Etapas:**
1. **Ordenação por Data**:
   - O dataframe foi filtrado para o `target_outlier` (cliente fora do padrão), e os dados foram ordenados pela coluna `datetime` para garantir que as análises sigam a sequência cronológica correta.
   - Isso nos permite observar como o consumo evolui ao longo do tempo e se há padrões temporais consistentes de consumo elevado.

2. **Identificação de Datas Únicas**:
   - Usamos o método `.unique()` na coluna de datas para listar todas as datas únicas em que há registros de consumo do outlier.
   - Isso facilita a análise dos dias em que o consumo foi registrado, permitindo identificar se o comportamento anômalo se concentra em datas específicas ou é distribuído de forma mais uniforme.

**Objetivo**:
- Através dessa análise detalhada, conseguimos observar o comportamento do outlier ao longo de diferentes períodos, o que pode ajudar a identificar padrões sazonais, eventos específicos ou falhas no sistema de medição.
- Essa investigação mais profunda pode ser usada para alertar os operadores de possíveis problemas ou anomalias no consumo de energia desse cliente.


# Modelo 2

Nossa abordagem busca melhorar a visualização dos consumos negativos, que foram muito proeminentes nas análises anteriores, expandindo o que apresentamos na última sprint. Para isso, segmentamos a base de dados em duas partes: indivíduos que apresentam consumo negativo e aqueles que não apresentam. Dentro de cada grupo, aplicamos técnicas de clusterização para identificar padrões que possam explicar os consumos negativos.

### Etapas da Análise

1. **Segmentação da Base de Dados:** A base foi dividida em dois subconjuntos: 
   - Pessoas com consumo negativo.
   - Pessoas sem consumo negativo.

2. **Clusterização Inicial com K-Means:**
   - Em cada subconjunto, aplicamos o algoritmo K-Means para identificar agrupamentos dentro das duas populações.
   - Utilizamos o método do cotovelo (Elbow Method) para determinar o número ideal de clusters para cada grupo.
   - Inicialmente, o K-Means não gerou agrupamentos suficientemente distintos para uma análise aprofundada.

3. **Ajuste do Número de Clusters:**
   - Diante dos resultados iniciais, aumentamos o número de clusters para verificar se uma divisão mais granular melhoraria a separação dos grupos.
   - Essa abordagem trouxe uma leve melhora na identificação de padrões, mas ainda não foi o suficiente para explicar completamente os consumos negativos.

4. **Identificação de Anomalias com Isolation Forest:**
   - Para complementar a análise, aplicamos o algoritmo Isolation Forest, que é especializado na detecção de anomalias.
   - Utilizamos o Grid Search para otimizar os hiperparâmetros do Isolation Forest, buscando o melhor ajuste para identificar comportamentos atípicos dentro dos clusters.

### Resultados e Próximos Passos

- A combinação de K-Means e Isolation Forest permitiu uma identificação mais refinada de padrões e anomalias entre os indivíduos com consumo negativo.
- A análise revelou que, em alguns casos, os consumos negativos podem ser atribuídos a comportamentos específicos que se destacam como outliers nos clusters.
- Os próximos passos incluem a análise mais detalhada dos clusters e das anomalias identificadas, com o objetivo de compreender as causas subjacentes dos consumos negativos e propor intervenções específicas.

Com essa abordagem, buscamos não apenas mapear os padrões de consumo, mas também entender as anomalias que surgem nos dados, oferecendo uma visão mais completa e direcionada para a resolução desses casos.

In [238]:
df2 = df.copy()

In [ ]:
df2 = df2.sort_values(by=["C_clientCode", "datetime"], ascending=True)

df2["match"] = df2.clientCode.eq(df2.clientCode.shift())

df2

In [ ]:
df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
x = df2["datetime"][1] - df2["datetime"][0]
print(x.total_seconds())

In [242]:
from datetime import timedelta
df2["time_diff_secs"] = None

df2["time_diff_secs"] = df2["match"].where(df2["match"] == False, df2["datetime"].diff())

df2["time_diff_secs"] = df2["time_diff_secs"].where(
    df2["match"] != False, timedelta(seconds=0).total_seconds(),
)

In [243]:
df2["time_diff_secs"] = df2["time_diff_secs"].apply(
    lambda x: x.total_seconds() if x != 0.0 else x
)

In [ ]:
# Agrupando por cliente e verificando se existe ao menos um valor negativo em 'measure_avg_consumption'
df2['tem_negativos'] = df2.groupby('C_clientCode')['measure_avg_consumption'].transform(lambda x: (x < 0).any())

# Exibindo o resultado
print(df2)



In [ ]:
import seaborn as sns

# Filtrando os dados para clientes com medições negativas
df_negativos = df2[df2['tem_negativos'] == True][['C_condCode', 'measure_avg_consumption']]
df_positivos = df2[df2['tem_negativos'] == False][['C_condCode', 'measure_avg_consumption']]

# Número de clusters definidos a partir dos resultados do Elbow e Silhueta
n_clusters_negativos = 7  # Defina o número de clusters ideal para clientes com medições negativas
n_clusters_positivos = 3  # Defina o número de clusters ideal para clientes sem medições negativas

# Aplicando o K-Means para clientes com medições negativas
kmeans_negativos = KMeans(n_clusters=n_clusters_negativos, random_state=42)
df_negativos['cluster'] = kmeans_negativos.fit_predict(df_negativos)

# Aplicando o K-Means para clientes sem medições negativas
kmeans_positivos = KMeans(n_clusters=n_clusters_positivos, random_state=42)
df_positivos['cluster'] = kmeans_positivos.fit_predict(df_positivos)

# Plotando os clusters para clientes com medições negativas
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.scatterplot(x='C_condCode', y='measure_avg_consumption', hue='cluster', data=df_negativos, palette='Set1')
plt.title('Clientes com Medições Negativas')
plt.xlabel('C_condCode')
plt.ylabel('Medição Média de Consumo')

# Plotando os clusters para clientes sem medições negativas
plt.subplot(1, 2, 2)
sns.scatterplot(x='C_condCode', y='measure_avg_consumption', hue='cluster', data=df_positivos, palette='Set2')
plt.title('Clientes sem Medições Negativas')
plt.xlabel('C_condCode')
plt.ylabel('Medição Média de Consumo')

# Exibindo os gráficos
plt.tight_layout()
plt.show()

# Exibindo os resultados dos clusters
print("Clusters para clientes com medições negativas:")
print(df_negativos)

print("\nClusters para clientes sem medições negativas:")
print(df_positivos)


In [ ]:
import seaborn as sns
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# Filtrando os dados para clientes com medições negativas
df_negativos = df2[df2['tem_negativos'] == True][['C_condCode', 'measure_avg_consumption']]
df_positivos = df2[df2['tem_negativos'] == False][['C_condCode', 'measure_avg_consumption']]

# Aplicando o Isolation Forest para clientes com medições negativas
iso_forest = IsolationForest(contamination=0.1, random_state=42)  # Define 5% dos dados como outliers
df_negativos['anomaly'] = iso_forest.fit_predict(df_negativos)

# O Isolation Forest retorna -1 para anomalias (outliers) e 1 para pontos normais
df_negativos['anomaly'] = df_negativos['anomaly'].apply(lambda x: 'Outlier' if x == -1 else 'Normal')

# Plotando os resultados do Isolation Forest para clientes com medições negativas
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.scatterplot(x='C_condCode', y='measure_avg_consumption', hue='anomaly', data=df_negativos, palette='Set1')
plt.title('Clientes com Medições Negativas (Anomalias)')
plt.xlabel('C_condCode')
plt.ylabel('Medição Média de Consumo')

# Aplicando o K-Means para clientes sem medições negativas (mantendo como K-Means para comparação)
n_clusters_positivos = 3  # Defina o número de clusters ideal para clientes sem medições negativas
kmeans_positivos = KMeans(n_clusters=n_clusters_positivos, random_state=42)
df_positivos['cluster'] = kmeans_positivos.fit_predict(df_positivos)

# Plotando os clusters para clientes sem medições negativas
plt.subplot(1, 2, 2)
sns.scatterplot(x='C_condCode', y='measure_avg_consumption', hue='cluster', data=df_positivos, palette='Set2')
plt.title('Clientes sem Medições Negativas')
plt.xlabel('C_condCode')
plt.ylabel('Medição Média de Consumo')

# Exibindo os gráficos
plt.tight_layout()
plt.show()

# Exibindo os resultados do Isolation Forest
print("Anomalias para clientes com medições negativas:")
print(df_negativos)

print("\nClusters para clientes sem medições negativas:")
print(df_positivos)


In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV

# Filtrando os dados para clientes com medições negativas
df_negativos = df2[df2['tem_negativos'] == True][['C_condCode', 'measure_avg_consumption']]

# Definindo o modelo de Isolation Forest
iso_forest = IsolationForest(random_state=42)

# Definindo o Grid de Hiperparâmetros para testar
param_grid = {
    'n_estimators': [50, 100],  # Testar 50, 100 e 200 árvores
    'max_samples': ['auto', 0.6],  # Auto, 60%, 80% ou 100% das amostras
    'contamination': [0.01, 0.05, 0.1],  # Testar 1%, 5% e 10% de contaminação (outliers)
    'max_features': [1.0, 0.5, 0.75]  # Testar todas as features, 50%, 75% das features
}

# Configurando o GridSearchCV
grid_search = GridSearchCV(
    estimator=iso_forest,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Métrica de avaliação
    cv=5,  # Validação cruzada com 5 folds
    verbose=1,  # Exibir progresso
    n_jobs=-1  # Usar todos os processadores disponíveis
)

# Rodando o GridSearch
grid_search.fit(df_negativos)

# Exibindo os melhores parâmetros encontrados
print("Melhores parâmetros encontrados pelo GridSearch:")
print(grid_search.best_params_)

# Exibindo o melhor modelo
best_model = grid_search.best_estimator_
print("Melhor modelo de Isolation Forest:")
print(best_model)

# Aplicando o melhor modelo aos dados
df_negativos['anomaly'] = best_model.predict(df_negativos)

# Convertendo -1 para "Outlier" e 1 para "Normal"
df_negativos['anomaly'] = df_negativos['anomaly'].apply(lambda x: 'Outlier' if x == -1 else 'Normal')

# Plotando os resultados
plt.figure(figsize=(8, 6))
sns.scatterplot(x='C_condCode', y='measure_avg_consumption', hue='anomaly', data=df_negativos, palette='Set1')
plt.title('Clientes com Medições Negativas (Anomalias com Isolation Forest)')
plt.xlabel('C_condCode')
plt.ylabel('Medição Média de Consumo')
plt.show()


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Função para calcular e plotar o método Elbow
def plot_elbow(df2, cluster_range, title):
    inertia = []
    
    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(df2)
        inertia.append(kmeans.inertia_)
    
    # Plot do método Elbow
    plt.plot(cluster_range, inertia, marker='o')
    plt.title(f'Método Elbow - {title}')
    plt.xlabel('Número de Clusters')
    plt.ylabel('Inércia (Soma dos Erros Quadráticos)')
    plt.show()

cluster_range = range(1, 10)
# Clientes sem medições negativas
print("\nClientes sem medições negativas:")
plot_elbow(df_positivos, cluster_range, 'Clientes sem Medições Negativas')


# Modelo 3

Assim como no primeiro modelo, nos motivamos a comparar um indivíduo contra a média de seu condomínio, mas dessa vez em dias específicos.
### 1. **Preprocessamento dos Dados**
   - **Conversão da Coluna de Tempo**: A coluna `timestamp` foi convertida para um formato de data e hora (`datetime`) para facilitar a manipulação dos dados.
   - **Filtro por Dia da Semana**: O código foi ajustado para filtrar os dados de consumo de acordo com o dia da semana escolhido.
   - **Ordenação dos Dados**: Os dados foram ordenados por condomínio (`condCode`), cliente (`clientCode`) e data (`datetime`) para garantir a consistência temporal das medições.
   - **Cálculo da Diferença de Tempo**: A diferença de tempo entre medições consecutivas foi calculada (`time_diff`), e os valores negativos ou nulos foram removidos para evitar problemas com cálculos de consumo.
### 2. **Tratamento de Valores de Consumo**
   - **Cálculo da Diferença de Consumo**: A coluna `measure_diff` foi criada para calcular a diferença de consumo entre medições consecutivas para cada cliente.
   - **Remoção de Valores Negativos**: Como valores negativos de consumo não fazem sentido no contexto do projeto, eles foram removidos do dataset.
   - **Cálculo do Consumo por Segundo**: A taxa de consumo por segundo foi calculada dividindo `measure_diff` por `time_diff`, criando a coluna `consumption_seconds`. Essa abordagem normaliza o consumo considerando a irregularidade nas medições temporais.
### 3. **Agrupamento de Consumo**
   - **Consumo Individual**: Para cada cliente, foi calculado o consumo total normalizado ao longo do dia, somando a coluna `consumption_seconds`. O resultado foi armazenado na coluna `total_individual_consumption`.
   - **Média de Consumo do Condomínio**: A média de consumo de todos os clientes dentro de cada condomínio, em um determinado dia, foi calculada e armazenada na coluna `mean_cond`.
### 4. **Método do Cotovelo (Elbow Method)**
   - O **método do cotovelo** foi aplicado para determinar o número ideal de clusters ao aplicar o algoritmo **K-Means**. A inércia (soma dos quadrados das distâncias dos pontos ao centróide mais próximo) foi calculada para diferentes valores de K (número de clusters).
   - Um gráfico foi gerado para identificar o ponto de "cotovelo", onde o ganho em separar os dados em mais clusters começa a diminuir, indicando o número ideal de clusters.
### 5. **Aplicação de K-Means**
   - Após determinar o número ideal de clusters pelo método do cotovelo, o algoritmo **K-Means** foi aplicado para agrupar os clientes com base no consumo total individual (`total_individual_consumption`) e na média de consumo do condomínio (`mean_cond`).
   - Os resultados da clusterização foram armazenados na coluna `cluster`, representando a qual grupo cada cliente pertence.
### 6. **Detecção de Anomalias com DBSCAN**
   - O algoritmo **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) foi utilizado para detectar anomalias no comportamento de consumo dos clientes.
   - Dois parâmetros principais foram ajustados:
     - **`eps` (raio de alcance)**: Determina a distância máxima para que dois pontos sejam considerados vizinhos.
     - **`min_samples`**: Define o número mínimo de clientes próximos necessários para formar um cluster.
   - Clientes cujos padrões de consumo estavam muito distantes dos demais foram classificados como **anomalias** e marcados com o rótulo `-1` pelo DBSCAN.
   - Um gráfico de **K-vizinho mais próximo (KNN Distance Plot)** foi utilizado para ajustar o valor de `eps`. O ponto onde a curva da distância começa a subir rapidamente foi identificado como o valor ideal para `eps`.
### 7. **Visualizações e Interpretação**
   - Várias visualizações foram geradas para facilitar a interpretação dos resultados, incluindo:
     - **Gráfico K-Vizinho Mais Próximo**: Utilizado para ajustar o parâmetro `eps` no DBSCAN.
     - **Gráfico do Método do Cotovelo**: Utilizado para determinar o número ideal de clusters para o K-Means.
     - **Gráficos de Dispersão (Scatter Plots)**: Visualizaram os clientes em relação ao consumo individual e ao consumo médio do condomínio, destacando as anomalias detectadas pelo DBSCAN.
### 8. **Análise Final e Conclusões**
   - A análise dos clusters ajudou a identificar grupos de clientes com comportamentos de consumo semelhantes, facilitando a interpretação dos padrões de consumo de energia.
   - O **DBSCAN** foi especialmente útil na detecção de **anomalias**, ou seja, clientes que apresentaram um comportamento de consumo muito diferente do esperado. Esses clientes foram marcados como "anomalias" e podem representar fraudes, erros de medição ou mudanças inesperadas no padrão de consumo.
   - A utilização de técnicas de clusterização e detecção de anomalias forneceu insights valiosos sobre o comportamento de consumo dos clientes, permitindo identificar padrões e possíveis problemas de forma eficiente.
Esse resumo cobre todas as alterações e melhorias feitas no projeto. Ele documenta de forma clara o que foi implementado e como as diferentes técnicas foram utilizadas para alcançar os resultados desejados. Caso precise de mais ajustes ou explicações detalhadas sobre algum ponto específico, estou à disposição!

In [113]:
df3 = df.copy()

In [ ]:
df3 = df3.sort_values(by=["C_clientCode", "datetime"], ascending=True)

df3["match"] = df3.clientCode.eq(df3.clientCode.shift())

df3

In [ ]:
df3.reset_index(drop=True, inplace=True)
df3

In [ ]:
x = df3["datetime"][1] - df3["datetime"][0]
print(x.total_seconds())

In [117]:
from datetime import timedelta
df3["time_diff_secs"] = None

df3["time_diff_secs"] = df3["match"].where(df3["match"] == False, df3["datetime"].diff())

df3["time_diff_secs"] = df3["time_diff_secs"].where(
    df3["match"] != False, timedelta(seconds=0).total_seconds(),
)

In [ ]:
df3

In [119]:
df3["time_diff_secs"] = df3["time_diff_secs"].apply(
    lambda x: x.total_seconds() if x != 0.0 else x
)

In [ ]:
df3

In [ ]:

# Cria uma coluna com o nome do dia da semana
df3['day_name'] = df3['datetime'].dt.day_name()

# Filtrar para o dia escolhido
df3 = df3[df3['day_name'] == 'Sunday']

# Garantir que os dados estão ordenados corretamente por condomínio (condCode), cliente (clientCode) e data
df3 = df3.sort_values(by=['condCode', 'clientCode', 'datetime'])

df3['time_diff'] = df3.groupby('clientCode')['datetime'].diff().dt.total_seconds()

# df3 = df3.dropna(subset=['measure_diff', 'time_diff'])

# Evitar divisões por zero ou valores negativos de tempo
df3 = df3[df3['time_diff'] > 0]

# Calcular a taxa de consumo (consumo por segundo) entre as medições
df3['consumption_seconds'] = df3['measure_diff'] / df3['time_diff']

# Agrupar por condCode, clientCode e dia da semana e calcular a diferença de consumo para cada grupo
individual_result = df3.groupby(['condCode', 'clientCode', 'day_name'])['consumption_seconds'].sum().reset_index(name='total_individual_consumption')

cond_result = df3.groupby(['condCode', 'day_name'])['consumption_seconds'].mean().reset_index(name='mean_cond')

df_result = individual_result.merge(cond_result, on=['condCode', 'day_name'], how='left')

print(df_result.head())

X = df_result[['total_individual_consumption', 'mean_cond']]

# X = X.dropna()

# Escalar os dados para a clusterização
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar o método do cotovelo para encontrar o número ideal de clusters
inertia = []  # Lista para armazenar as inércias
K = range(1, 11)  # Testando de 1 até 10 clusters
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

# Plotar o gráfico do método do cotovelo
plt.figure(figsize=(8, 6))
plt.plot(K, inertia, 'bo-', markersize=8)
plt.xlabel('Número de Clusters (K)', fontsize=14)
plt.ylabel('Inércia', fontsize=14)
plt.title('Método do Cotovelo para Encontrar o Número Ideal de Clusters', fontsize=16)
plt.grid(True)
plt.show()


In [ ]:
# Aplicar K-Means com o número de clusters ideal após analisar o gráfico do método do cotovelo
kmeans = KMeans(n_clusters=2, random_state=42)  # Substitua 3 pelo número ideal encontrado
kmeans.fit(X_scaled)

# Adicionar os rótulos de cluster ao DataFrame original
df_result['cluster'] = kmeans.labels_

cluster_centers = kmeans.cluster_centers_

# Reverter a escala dos centros para o espaço original
cluster_centers_og = scaler.inverse_transform(cluster_centers)

# Exibir as primeiras linhas para verificar o resultado dos clusters
print(df_result.head())

In [ ]:
cluster_1 = df_result[df_result['cluster'] == 1]

print(cluster_1.head())

In [ ]:
from sklearn.cluster import DBSCAN
# Configurar DBSCAN
dbscan = DBSCAN(eps=0.016, min_samples=3)  # eps define o raio de alcance para formar um cluster, min_samples define o número mínimo de pontos para formar um cluster
# Aplicar DBSCAN nos dados de consumo
X_dbscan = df_result[['total_individual_consumption', 'mean_cond']]
df_result['dbscan_cluster'] = dbscan.fit_predict(X_dbscan)
# DBSCAN atribui o valor -1 para anomalias
df_result['dbscan_anomalia'] = df_result['dbscan_cluster'].apply(lambda x: 'Anomaly' if x == -1 else 'Normal')
# Visualizar os resultados
print("Anomalias detectadas pelo DBSCAN:")
print(df_result[df_result['dbscan_anomalia'] == 'Anomaly'])
# Visualizar as anomalias detectadas pelo DBSCAN
plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_individual_consumption', y='mean_cond', hue='dbscan_anomalia', data=df_result, palette='Set1', s=100)
plt.title('Anomalias Detectadas pelo DBSCAN')
plt.xlabel('Consumo Total Individual')
plt.ylabel('Média do Consumo do Condomínio')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
# Definir o número de vizinhos (min_samples)
min_samples = 5
# Ajustar o modelo Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=min_samples)
nbrs = neigh.fit(X_dbscan)
distances, indices = nbrs.kneighbors(X_dbscan)
# Ordenar as distâncias dos vizinhos mais próximos
distances = np.sort(distances[:, min_samples-1])
# Plotar o gráfico da distância K-vizinho mais próximol

plt.figure(figsize=(8, 6))
plt.plot(distances)
plt.xlabel('Pontos de Dados Ordenados', fontsize=14)
plt.ylabel(f'Distância para o {min_samples}º Vizinho Mais Próximo', fontsize=14)
plt.title(f'Gráfico K-Vizinho Mais Próximo para Determinar eps (min_samples={min_samples})', fontsize=16)
plt.grid(True)
plt.ylim(0, 0.1)
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score
# Testar diferentes valores de eps e min_samples
eps_values = [0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.025]  # Exemplo de valores de eps
min_samples_values = [3, 5, 7]  # Exemplo de valores de min_samples
best_score = -1
best_params = None
# Iterar sobre combinações de eps e min_samples
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_dbscan)
        # Checar se mais de um cluster foi formado
        if len(set(labels)) > 1:
            # Calcular o score de Silhouette
            score = silhouette_score(X_dbscan, labels)
            print(f'eps: {eps}, min_samples: {min_samples}, Silhouette Score: {score}')
            # Armazenar a melhor configuração de parâmetros
            if score > best_score:
                best_score = score
                best_params = (eps, min_samples)
print(f'Melhores parâmetros encontrados: eps={best_params[0]}, min_samples={best_params[1]}, Silhouette Score={best_score}')

In [ ]:
# Plotar os clusters após aplicar o K-Means
plt.figure(figsize=(10, 6))

# Scatter plot dos clusters formados
plt.scatter(df_result['total_individual_consumption'], df_result['mean_cond'], 
            c=df_result['cluster'], cmap='viridis', s=50, label='Clientes')

# Plotar os centros dos clusters
plt.scatter(cluster_centers_og[:, 0], cluster_centers_og[:, 1], 
            c='red', s=300, marker='X', edgecolor='black', label='Centros dos Clusters')

# Rótulos e título
plt.colorbar(label='Cluster')
plt.xlabel('Consumo Total Individual', fontsize=14)
plt.ylabel('Média do Consumo do Condomínio', fontsize=14)
plt.title('Distribuição dos Clusters de Consumo', fontsize=16)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Contar a quantidade de clientes em cada cluster
cluster_counts = df_result['cluster'].value_counts()
print("Quantidade de clientes em cada cluster:")
print(cluster_counts)
# Calcular as estatísticas dos clusters
cluster_stats = df_result.groupby('cluster').agg({
    'total_individual_consumption': ['mean', 'std'],
    'mean_cond': ['mean', 'std']
}).reset_index()

print(cluster_stats)

# Definir um limite para identificar anomalias
def identify_anomaly(row, stats):
    cluster = row['cluster']
    mean_individual = stats.loc[stats['cluster'] == cluster, ('total_individual_consumption', 'mean')].values[0]
    std_individual = stats.loc[stats['cluster'] == cluster, ('total_individual_consumption', 'std')].values[0]
 
    if row['total_individual_consumption'] > mean_individual + 2 * std_individual:
        return 'Acima da media'
    elif row['total_individual_consumption'] < mean_individual - 2 * std_individual:
        return 'Abaixo da media'
    else:
        return 'Normal'
# Aplicar a função para identificar anomalias
df_result['anomaly'] = df_result.apply(identify_anomaly, axis=1, stats=cluster_stats)

print("Clientes com possíveis anomalias:")
print(df_result[df_result['anomaly'] != 'Normal'])

In [ ]:
from sklearn.ensemble import IsolationForest
# Configurar Isolation Forest
iso_forest = IsolationForest(bootstrap = False, contamination = 0.01, max_features = 1, max_samples = 0.1, n_estimators = 12, random_state=42)  # Contamination define a proporção esperada de anomalias
# Aplicar Isolation Forest nos dados de consumo
X_iso = df_result[['total_individual_consumption', 'mean_cond']]
df_result['iso_forest_anomalia'] = iso_forest.fit_predict(X_iso)
# -1 indica anomalias, 1 indica pontos normais
df_result['iso_forest_anomalia'] = df_result['iso_forest_anomalia'].map({1: 'Normal', -1: 'Anomalia'})
# Visualizar os resultados  
print("Anomalias detectadas pelo Isolation Forest:")
print(df_result[df_result['iso_forest_anomalia'] == 'Anomalia'])
# Visualizar as anomalias detectadas pelo Isolation Forest
plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_individual_consumption', y='mean_cond', hue='iso_forest_anomalia', data=df_result, palette='coolwarm', s=100)
plt.title('Anomalias Detectadas pelo Isolation Forest')
plt.xlabel('Consumo Total Individual')
plt.ylabel('Média do Consumo do Condomínio')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV
import numpy as np

# Definir o espaço de busca dos hiperparâmetros
param_grid = {
    'n_estimators': [12, 25, 50, 100, 200, 400, 800],  # Número de árvores
    'max_samples': [0.1,0.25,0.5, 0.75, 1.0],  # Fração das amostras
    'contamination': np.linspace(start=0.01, stop=0.05, num=10),  # Valores de contaminação
    'max_features': [1, 2],  # Número de features
    'bootstrap': [False, True],  # Se deve usar bootstrap
}

# Instanciar o Isolation Forest
iso_forest = IsolationForest(random_state=42)

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=iso_forest, param_grid=param_grid, scoring='f1', cv=5, verbose=2, n_jobs=-1)

# Aplicar o GridSearchCV nos dados
X_iso = df_result[['total_individual_consumption', 'mean_cond']]
grid_search.fit(X_iso)

# Verificar os melhores parâmetros encontrados
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Melhores parâmetros encontrados:", best_params)
print("Melhor pontuação F1 obtida:", best_score)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Configurar o RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=iso_forest, param_distributions=param_grid, n_iter=50, scoring='f1', cv=5, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(X_iso)

# Verificar os melhores parâmetros encontrados
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Melhores parâmetros encontrados:", best_params)
print("Melhor pontuação F1 obtida:", best_score)

In [ ]:
plt.figure(figsize=(25, 15))
# Plotar o consumo do outlier
plt.scatter(
    df3[(df3["C_clientCode"] == 907) & (df3["datetime"] > "2024-05")]["datetime"],
    df3[(df3["C_clientCode"] == 907) & (df3["datetime"] > "2024-05")]["measure_diff"],
    label="Consumo do outlier",
    color="blue",
)
# Plotar o consumo médio do condomínio
plt.scatter(
    df3[(df3["C_condCode"] == 27) & (df3["datetime"] > "2024-05")]["datetime"],
    df3[(df3["C_condCode"] == 27) & (df3["datetime"] > "2024-05")]["measure_avg_consumption"],
    label="Consumo médio do condomínio",
    color="orange",
)
# Título e rótulos com tamanho de fonte ajustado
plt.title("Consumo do outlier vs Consumo do condomínio", fontsize=20)
plt.xlabel("Date", fontsize=20)
plt.ylabel("Consumption", fontsize=20)

# Configuração dos ticks
plt.xticks(rotation=90, fontsize=12)  # Aumentando o tamanho dos valores no eixo X
plt.yticks(fontsize=12)  # Aumentando o tamanho dos valores no eixo Y


# Adicionar legenda com tamanho de fonte ajustado
plt.legend(fontsize=25)

# Ajustar o layout
plt.tight_layout()

# Mostrar o gráfico
plt.show()

In [ ]:
# Filtrar os dados do Cluster 1
cluster_1 = df_result[df_result['cluster'] == 1]
# Exibir as primeiras linhas dos dados do Cluster 1
print("Clientes pertencentes ao Cluster 1:")
print(cluster_1.head())
# Se quiser ver todos os clientes do Cluster 1, pode remover o .head() e usar .to_string() para mostrar todos
print("Todos os clientes pertencentes ao Cluster 1:")
print(cluster_1.to_string())  # Usar to_string() para mostrar o DataFrame inteiro

In [ ]:
# Gera um gráfico de dispersão para clientes pertencentes ao Cluster 1.

plt.figure(figsize=(10, 6))
sns.scatterplot(x='total_individual_consumption', y='mean_cond', data=cluster_1, s=100)
plt.title('Clientes pertencentes ao Cluster 1')
plt.xlabel('Consumo Total Individual')
plt.ylabel('Média do Consumo do Condomínio')
plt.grid(True)
plt.show()

In [ ]:
# Verificar se há valores negativos em 'measure_diff'
negativos_measure_diff = df3[df3['measure_diff'] < 0]
# Exibir os valores negativos, se existirem
if not negativos_measure_diff.empty:
    print("Valores negativos encontrados em 'measure_diff':")
    print(negativos_measure_diff[['C_clientCode', 'C_condCode', 'measure_diff']].to_string(index=False))
else:
    print("Não há valores negativos em 'measure_diff'.")

In [ ]:
negativos_measure_diff['C_clientCode'].nunique()


In [ ]:
negativos_measure_diff['C_condCode'].nunique()

In [ ]:
df3['C_condCode'].nunique()